# 1. Imports

In [2]:
from __future__ import annotations

import os
from pathlib import Path
from datetime import datetime
from typing import Iterable, Dict, Any, List

import pandas as pd
from neo4j import GraphDatabase, basic_auth
from neo4j.exceptions import ServiceUnavailable, TransientError
from dotenv import load_dotenv

# 2. Configurations

In [16]:
project_dir = Path().cwd()
dotenv_path = project_dir / '.env'

# Checks if the .env file exists at the constructed path before loading.
if dotenv_path.exists():
    # Loads the environment variables from the found .env file.
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Loaded .env file from: {dotenv_path}")
else:
    print(f".env file not found at: {dotenv_path}")

load_dotenv(dotenv_path=dotenv_path)
URI  = os.getenv("NEO4J_URI") 
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

# Function to execute a query
def run_query(driver, query, **params):
    """Executes a Cypher query and returns the result."""
    with driver.session() as session:
        result = session.run(query, **params)
        return [record for record in result]

# Establish the connection to the database
try:
    driver = GraphDatabase.driver(URI, auth=AUTH)
    driver.verify_connectivity()
    print("✅ Connection to Neo4j successful!")
except Exception as e:
    print(f"❌ Failed to connect to Neo4j: {e}")

Loaded .env file from: /Users/stahlma/Desktop/01_Studium/10_Seminar/causal-rl-ev-review/.env
✅ Connection to Neo4j successful!


# 3. Uniqueness Constraint

In [17]:
# Create a uniqueness constraint on Paper nodes
print("Creating constraint for Paper nodes...")
paper_constraint_query = "CREATE CONSTRAINT paper_id_unique IF NOT EXISTS FOR (p:Paper) REQUIRE p.paperId IS UNIQUE;"
run_query(driver, paper_constraint_query)

# Create a uniqueness constraint on Author nodes
print("Creating constraint for Author nodes...")
author_constraint_query = "CREATE CONSTRAINT author_id_unique IF NOT EXISTS FOR (a:Author) REQUIRE a.authorId IS UNIQUE;"
run_query(driver, author_constraint_query)

# Constraint for Query nodes
print("Creating constraint for Query nodes...")
query_constraint = "CREATE CONSTRAINT query_id_unique IF NOT EXISTS FOR (q:Query) REQUIRE q.queryId IS UNIQUE;"
run_query(driver, query_constraint)

# Constraint for FieldOfStudy nodes
print("Creating constraint for FieldOfStudy nodes...")
fos_constraint = "CREATE CONSTRAINT concept_id_unique IF NOT EXISTS FOR (f:FieldOfStudy) REQUIRE f.conceptId IS UNIQUE;"
run_query(driver, fos_constraint)

# Constraint for Venue nodes
print("Creating constraint for Venue nodes...")
venue_constraint = "CREATE CONSTRAINT venue_id_unique IF NOT EXISTS FOR (v:Venue) REQUIRE v.venueId IS UNIQUE;"
run_query(driver, venue_constraint)

print("Constraints created successfully.")

Creating constraint for Paper nodes...
Creating constraint for Author nodes...
Creating constraint for Query nodes...
Creating constraint for FieldOfStudy nodes...
Creating constraint for Venue nodes...
Constraints created successfully.


# 4. Load Nodes

In [19]:
# Load the papers CSV
papers_df = pd.read_csv('./data/processed/Papers.csv')
# Convert to a list of dictionaries for the query
papers_records = papers_df.to_dict('records')

# Cypher query to create Paper nodes from a list of records
create_papers_query = """
UNWIND $records AS row
MERGE (p:Paper {paperId: row.paper_id})
SET
    p.title = row.title,
    p.abstract = row.abstract,
    p.year = toInteger(row.year),
    p.publicationDate = row.publication_date,
    p.doi = row.doi,
    p.s2Url = row.s2_url
"""

# Send the data in batches of 5000
print(f"Loading {len(papers_records)} Paper nodes...")
batch_size = 5000
for i in range(0, len(papers_records), batch_size):
    batch = papers_records[i:i + batch_size]
    run_query(driver, create_papers_query, records=batch)
print("✅ Paper nodes loaded.")

# --- Load Author Nodes ---
authors_df = pd.read_csv('./data/processed/Authors.csv')
authors_records = authors_df.to_dict('records')

create_authors_query = """
UNWIND $records AS row
MERGE (a:Author {authorId: row.author_id})
SET a.name = row.name
"""

print(f"\nLoading {len(authors_records)} Author nodes...")
for i in range(0, len(authors_records), batch_size):
    batch = authors_records[i:i + batch_size]
    run_query(driver, create_authors_query, records=batch)
print("✅ Author nodes loaded.")

# --- Load Query Nodes ---
queries_df = pd.read_csv('./data/processed/Queries.csv')
queries_records = queries_df.to_dict('records')

create_queries_query = """
UNWIND $records AS row
MERGE (q:Query {queryId: row.query_id})
SET q.name = row.name
"""

print(f"\nLoading {len(queries_records)} Query nodes...")
run_query(driver, create_queries_query, records=queries_records)
print("✅ Query nodes loaded.")


# --- Load FieldOfStudy Nodes ---
fos_df = pd.read_csv('./data/processed/FieldsOfStudy.csv')
fos_records = fos_df.to_dict('records')

create_fos_query = """
UNWIND $records AS row
MERGE (f:FieldOfStudy {conceptId: row.concept_id})
SET f.name = row.name
"""

print(f"\nLoading {len(fos_records)} FieldOfStudy nodes...")
run_query(driver, create_fos_query, records=fos_records)
print("✅ FieldOfStudy nodes loaded.")


# --- Load Venue Nodes ---
venues_df = pd.read_csv('./data/processed/Venues.csv')
venues_records = venues_df.to_dict('records')

create_venues_query = """
UNWIND $records AS row
MERGE (v:Venue {venueId: row.venue_id})
SET v.name = row.name, v.type = row.type, v.url = row.url
"""

print(f"\nLoading {len(venues_records)} Venue nodes...")
run_query(driver, create_venues_query, records=venues_records)
print("✅ Venue nodes loaded.")

Loading 2055 Paper nodes...
✅ Paper nodes loaded.

Loading 7004 Author nodes...
✅ Author nodes loaded.

Loading 8 Query nodes...
✅ Query nodes loaded.

Loading 14 FieldOfStudy nodes...
✅ FieldOfStudy nodes loaded.

Loading 932 Venue nodes...
✅ Venue nodes loaded.


# 5. Load Relationships

In [20]:
batch_size = 5000

# --- Load Authorship Relationships ---
authorship_df = pd.read_csv('./data/processed/Authorship.csv')
authorship_records = authorship_df.to_dict('records')

create_authorship_query = """
UNWIND $records AS row
MATCH (a:Author {authorId: row.author_id})
MATCH (p:Paper {paperId: row.paper_id})
MERGE (a)-[r:AUTHORED {position: toInteger(row.position)}]->(p)
"""

print(f"\nLoading {len(authorship_records)} AUTHORED relationships...")
for i in range(0, len(authorship_records), batch_size):
    batch = authorship_records[i:i + batch_size]
    run_query(driver, create_authorship_query, records=batch)
print("✅ AUTHORED relationships loaded.")


# --- Load Cites Relationships ---
cites_df = pd.read_csv('./data/processed/Cites.csv')
# Rename columns to avoid ambiguity in the query
cites_df.rename(columns={'src_paper_id': 'sourceId', 'dst_paper_id': 'destinationId'}, inplace=True)
cites_records = cites_df.to_dict('records')

create_cites_query = """
UNWIND $records AS row
MATCH (source:Paper {paperId: row.sourceId})
MATCH (destination:Paper {paperId: row.destinationId})
MERGE (source)-[:CITES]->(destination)
"""

print(f"\nLoading {len(cites_records)} CITES relationships...")
for i in range(0, len(cites_records), batch_size):
    batch = cites_records[i:i + batch_size]
    run_query(driver, create_cites_query, records=batch)
print("✅ CITES relationships loaded.")

# --- Load PUBLISHED_IN Relationships ---
published_in_df = pd.read_csv('./data/processed/PublishedIn.csv')
published_in_records = published_in_df.to_dict('records')

create_published_in_query = """
UNWIND $records AS row
MATCH (p:Paper {paperId: row.paper_id})
MATCH (v:Venue {venueId: row.venue_id})
MERGE (p)-[:PUBLISHED_IN]->(v)
"""
print(f"\nLoading {len(published_in_records)} PUBLISHED_IN relationships...")
for i in range(0, len(published_in_records), batch_size):
    batch = published_in_records[i:i + batch_size]
    run_query(driver, create_published_in_query, records=batch)
print("✅ PUBLISHED_IN relationships loaded.")


# --- Load FOUND_BY Relationships (FromQuery.csv) ---
from_query_df = pd.read_csv('./data/processed/FromQuery.csv')
from_query_records = from_query_df.to_dict('records')

create_from_query_query = """
UNWIND $records AS row
MATCH (p:Paper {paperId: row.paper_id})
MATCH (q:Query {queryId: row.query_id})
MERGE (p)-[:FOUND_BY]->(q)
"""
print(f"\nLoading {len(from_query_records)} FOUND_BY relationships...")
for i in range(0, len(from_query_records), batch_size):
    batch = from_query_records[i:i + batch_size]
    run_query(driver, create_from_query_query, records=batch)
print("✅ FOUND_BY relationships loaded.")


# --- Load HAS_FIELD Relationships ---
has_field_df = pd.read_csv('./data/processed/HasField.csv')
has_field_records = has_field_df.to_dict('records')

create_has_field_query = """
UNWIND $records AS row
MATCH (p:Paper {paperId: row.paper_id})
// The conceptId was created with a prefix and formatted name, so we recreate it here to match
MATCH (f:FieldOfStudy {conceptId: "F:" + toLower(replace(row.field_of_study, " ", "-"))})
MERGE (p)-[:HAS_FIELD]->(f)
"""
print(f"\nLoading {len(has_field_records)} HAS_FIELD relationships...")
for i in range(0, len(has_field_records), batch_size):
    batch = has_field_records[i:i + batch_size]
    run_query(driver, create_has_field_query, records=batch)
print("✅ HAS_FIELD relationships loaded.")


# --- Load SIMILAR_TO Relationships ---
similarity_df = pd.read_csv('./data/processed/Similarity.csv')
similarity_records = similarity_df.to_dict('records')

create_similarity_query = """
UNWIND $records AS row
MATCH (p1:Paper {paperId: row.src_paper_id})
MATCH (p2:Paper {paperId: row.dst_paper_id})
// We use an undirected relationship here because similarity is mutual
MERGE (p1)-[r:SIMILAR_TO]-(p2)
SET r.score = toFloat(row.score), r.method = row.method
"""
print(f"\nLoading {len(similarity_records)} SIMILAR_TO relationships...")
for i in range(0, len(similarity_records), batch_size):
    batch = similarity_records[i:i + batch_size]
    run_query(driver, create_similarity_query, records=batch)
print("✅ SIMILAR_TO relationships loaded.")


Loading 7845 AUTHORED relationships...
✅ AUTHORED relationships loaded.

Loading 805 CITES relationships...
✅ CITES relationships loaded.

Loading 1947 PUBLISHED_IN relationships...
✅ PUBLISHED_IN relationships loaded.

Loading 2055 FOUND_BY relationships...
✅ FOUND_BY relationships loaded.

Loading 1585 HAS_FIELD relationships...
✅ HAS_FIELD relationships loaded.

Loading 1600 SIMILAR_TO relationships...
✅ SIMILAR_TO relationships loaded.
